# NOWCASTING RADAR RAINMAP

Original version by Gabriel Colas : https://www.kaggle.com/code/gabrielcolas/radar-nowcasting

**/!\ To use it correctly on Kaggle, choose a GPU accelerator, and switch Internet on in the options**

The goal of this project is to use Radar Data from meteonet as an image and to predict the movement of a rainmap radar using deep learning. <br>
In the Meteonet data, for each radar data type, you will find one archive per month, each one sliced in periods of 10 or 11 days (each month is separated in 3 files). <br> For this notebook we will forecast one hour data and use a 15 minuts step. <br> <br> So first our goal is to process those raw data and get in the order: <br> 
* Pick data for a 15 minuts step
* Sequence of 6 rainmap for learning + 1 rainmap as label
* 256 * 256 map

* Repeat predict for 2 more predictions images by adding previous predictions

![](https://weatheregg.com/wp-content/uploads/2018/01/doppler-radar-map-weathergov.png)

In [ ]:
# IMPORT THE LIBRARIES
import os
import numpy as np
from datetime import datetime, timedelta

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
# from keras.optimizers import *


In [ ]:
# LOCATION OF THE TRAINING DATA
directory = '/kaggle/input/meteonet/'
directory_save = '/kaggle/working/'
zone = "NW"

In [ ]:
fname_coords = os.path.join(directory,'Radar_coords/Radar_coords',f'radar_coords_{zone}.npz')
coords = np.load(fname_coords, allow_pickle=True)

# Updated version : already centered
lat = coords['lats']
lon = coords['lons']

<h3> LOAD THE USEFULL FUNCTIONS FOR ANALYSIS </h3>

In [ ]:
# LOAD ANY RAINFALL RADAR PICKLE
def load_fichier(year,part_month,month):
    directory = '/kaggle/input/meteonet/'
    zone = "NW"
    fname = os.path.join(directory, f'{zone}_rainfall_{str(year)}/{zone}_rainfall_{str(year)}/rainfall-{zone}-{str(year)}-{str(month).zfill(2)}/rainfall-{zone}-{str(year)}-{str(month).zfill(2)}/rainfall_{zone}_{str(year)}_{str(month).zfill(2)}.{str(part_month)}.npz')
    pickle = np.load(fname, allow_pickle=True)
    return pickle

In [ ]:
# convert a date to an index, corresponding to an initial date
def indice_date(date_init, date_value):    
    Diff_year = date_value.year - date_init.year
    Diff_Month = date_value.month - date_init.month
    Diff_Day = date_value.day - date_init.day
    Diff_Hour = date_value.hour - date_init.hour
    Diff_Minute = date_value.minute - date_init.minute    
    indice = Diff_Day*24*12 + Diff_Hour*12 + Diff_Minute/Step_data
    
    return {'Diff_year' : Diff_year , 'Diff_Month' : Diff_Month, 'indice' : indice}


# Convert the given date of the pickle to the missing index.
def indice_miss_date(pickle):
    indice_tab= np.zeros(pickle['miss_dates'].shape[0],dtype=int)
    print("This is the missing date in the data: ",pickle['miss_dates'])
    
    compteur = 0
    for miss_date in pickle['miss_dates']:
        print('miss_date : ', miss_date)
        dico_i = indice_date(pickle['dates'][0], miss_date)
        indice_abs = int(dico_i['indice']) / dico_i['indice']
        # We raise error because for now we don't handle that kind of situation
        if dico_i['Diff_year'] != 0 or dico_i['Diff_Month'] != 0:
            raise NameError('There is a difference in month or year !')
        if indice_abs != 1:
            raise NameError('Indice is not a integer')
        else:
            indice_tab[compteur] = int(dico_i['indice'])
        compteur += 1
    return indice_tab


# cut the input data by the step we have consider
def cut_timestep(pickle, Step_minute):    
    print("We make a step every ", Step, "indices")
    Val_selec = np.arange(0,len(pickle['dates']), Step) # Every  'Step' index corresponding to 'Step_minute' minutes
    return pickle['data'][Val_selec,:,:], pickle['dates'][Val_selec]


# If Missing values.
def cut_timestep_miss(pickle,Echeance_minute,Step_minute) :
    
    Qinit = (Step * Tot) - Step
    Q1= Step * Tot
  
    indice_tab = indice_miss_date(pickle) # LOAD indice_miss_date function to get the missing indices 
    
    Tab_index = []    
    valeur_init = 0
    
    # last theorical index if no missing datas between begin and end times
    last_index = int(indice_date(pickle["dates"][0], pickle['dates'][-1])['indice'])
    
    for indice in indice_tab:
       
        Sequence = np.arange(valeur_init , indice , Step , dtype=int)
        Tab_index.append(Sequence[ : Tot *(len(Sequence) // Tot)])
    
        # We can begin to find the next sequence just after the next index
        # which must be a multiple of the Step
        i = 0
        while (((indice + i + 1) % Step) > 0) and (indice + i + 1 <= last_index):
            i += 1
        valeur_init = indice + i + 1   
       

    Sequence = np.arange(valeur_init , last_index + 1 , Step , dtype = int)
    Tab_index.append(Sequence[: Tot * (len(Sequence) // Tot)])

    return Tab_index


In [ ]:
def cut_data_and_coord(data, Pixel, lat,lon, lat_edge = 49.5, lon_edge = -2.5):    
    lat_only=lat[:,1]
    lon_only=lon[1,:]
    Temp_lat = np.where(lat_only < lat_edge)
    Temp_lon = np.where(lon_only < lon_edge)
    Lat_cut_index = Temp_lat[0][:Pixel]
    Lon_cut_index = Temp_lon[0][:Pixel]
    data_cut = data[:,Lat_cut_index]
    data_cut= data_cut[:,:,Lon_cut_index]
    return data_cut

def cut_timestep_miss2(pickle, Echeance_minute, Step_minute):
    Tab = cut_timestep_miss(pickle, Echeance_minute, Step_minute)
    # all the valid sequence indexes
    Tab_tot = np.concatenate(Tab)
    
    # but we have to make a conversion for missing dates indexes
    
    time_init = pickle['dates'][0]
    time_final = pickle['dates'][-1]
    
    # list all the theorical times if there was no missing data
    liste_t = np.arange(time_init, time_final + timedelta(minutes=Step_data), timedelta(minutes = Step_data)).astype(datetime)
   
    Tab_tot2 = np.empty(len(Tab_tot), dtype = int)
    
    # search the indexes of the pickle dataset corresponding with the indexes of the Tab_tot
    for i, indice in enumerate(Tab_tot):
        time_indice = time_init + indice * timedelta(minutes = Step_data)
        Tab_tot2[i] = np.where(pickle['dates'] == time_indice)[0][0]
        
    
    return pickle['data'][Tab_tot2, :, :], pickle['dates'][Tab_tot2]
    # it's possible to make another conversion method by just shifting indexes by one step every missing index
    

In [ ]:
def XYTRAIN(data_process):
    X_Train = np.zeros((0, Pixel, Pixel, Entrainement))
    Y_Train = np.zeros((0, Pixel, Pixel, Prediction))
    DATA_X = np.zeros((1, Pixel, Pixel, Entrainement))
    DATA_Y = np.zeros((1, Pixel, Pixel,Prediction))
    pas_X = 0
    pas_Y = Entrainement - 1
    print("Valeur max:", int(data_process.shape[0] / Tot) )
    for globale in range(int(data_process.shape[0] / Tot)):
        for X in range(1, Entrainement + 1):
            DATA_X[0, :, :, X - 1] = data_process[pas_X + X - 1, :, :]
        for Y in range(1, Prediction + 1):
            DATA_Y[0, :, :, Y-1] = data_process[pas_Y + Y, :, :]
        pas_X += Tot
        pas_Y += Tot
        X_Train = np.append(X_Train, DATA_X, axis = 0)
        Y_Train = np.append(Y_Train, DATA_Y, axis = 0)
        DATA_X = np.zeros((1, Pixel, Pixel, Entrainement))
        DATA_Y = np.zeros((1, Pixel, Pixel, Prediction))
    return X_Train, Y_Train

In [ ]:
def COUNT_RAIN_SITUATION(X_TRAIN, Y_TRAIN, rain_limit):
    Count_X = 0
    Count_Y = 0
    Tab_delete = []
    Count = 0
    for X in X_TRAIN:
        Temp_x = np.count_nonzero(X > rain_limit)
        if (Temp_x <= 200):
            Count_X += 1
            Tab_delete.append(Count)
        Count += 1
        
    for Y in Y_TRAIN:
        Temp_y = np.count_nonzero(Y > rain_limit)
        if Temp_y <= 200:
            Count_Y += 1
            
    X_TRAIN = np.delete(X_TRAIN, Tab_delete,axis = 0)
    Y_TRAIN = np.delete(Y_TRAIN, Tab_delete,axis = 0)
        
    return(X_TRAIN, Y_TRAIN, Count_X, Count_Y)

# SETTINGS

In [ ]:
# CONFIGURATION DATA
# CHOOSE THE STEP AND THE ECHEANCE
Echeance_minute = 45
Step_minute = 15
Pixel = 256
rain_limit = 0

In [ ]:
Step_data = 5  # step in the original data
Step = Step_minute // Step_data
# number of images to train and to predict
Prediction = Echeance_minute // Step_minute
Entrainement = Prediction + 3 
Tot = Entrainement + Prediction

In [ ]:
# GLOBAL CODE FOR GIVEN THE DATASET TO LEARN THE MODEL
month = [1, 2, 3]
part = [1, 2, 3]
year = 2016

# PREPARE TRAIN DATA

In [ ]:
## LOOP FOR TRAIN DATA
init = 0
for m in month:
    for p in part:
        pickle = load_fichier(year, p, m)
        print("This is the part " ,p)
    
        if pickle["miss_dates"].shape[0] == 0:
            print("There is no missing data in this chunk")
            data_radar,dates_radar = cut_timestep(pickle, Step_minute)
            print(data_radar.shape)
            data_cut = cut_data_and_coord(data_radar, Pixel, lat, lon)
            print( data_cut.shape)            
            X_TRAIN, Y_TRAIN = XYTRAIN(data_cut)
            print("Shape of the temporary train data", X_TRAIN.shape, Y_TRAIN.shape)
        
        else:
            print("There is miss dates")
            data_radar,dates_radar = cut_timestep_miss2(pickle, Echeance_minute, Step_minute)
            print(data_radar.shape)
            data_cut = cut_data_and_coord(data_radar, Pixel, lat, lon)
            print(data_cut.shape)            
            X_TRAIN, Y_TRAIN = XYTRAIN(data_cut)
            print("Shape of the temporary train data", X_TRAIN.shape, Y_TRAIN.shape)
        
        if (init == 0):
            X_TEMP = X_TRAIN
            Y_TEMP = Y_TRAIN
            print("STEP N°1: ", X_TEMP.shape, Y_TEMP.shape)
        elif ( init == 1):
            X_train = np.append(X_TEMP, X_TRAIN, axis = 0)
            Y_train = np.append(Y_TEMP, Y_TRAIN, axis = 0)
            print("STEP N°2: ",X_train.shape,Y_train.shape)
        else:
            X_train = np.append(X_train, X_TRAIN,axis = 0)
            Y_train = np.append(Y_train, Y_TRAIN,axis = 0)
            print("OTHER STEP : ", X_train.shape, Y_train.shape)
        init += 1
        print("----------------------------------------------------------------------")
        
print('End of Train')
del(X_TEMP, Y_TEMP, X_TRAIN, Y_TRAIN, data_cut, data_radar)

In [ ]:
# TRAIN ONLY ON IMAGE SEQUENCES WITH RAINS
X_train, Y_train, Count_X, Count_Y = COUNT_RAIN_SITUATION(X_train, Y_train, rain_limit)
print(X_train.shape, Y_train.shape, Count_X, Count_Y)


In [ ]:
# We make only 1 image prediction at one time
Y_train1 = Y_train[:, :, :, 0:1]

# U_NET NETWORKS

In [ ]:
class LRA(keras.callbacks.Callback):
    def __init__(self, model, initial_learning_rate, gamma, power):
        super(LRA, self).__init__()
        self.initial_learning_rate = initial_learning_rate
        self.gamma = gamma
        self.power = power
        self.model = model # model is your compiled model
    def on_train_begin(self, logs=None):
        tf.keras.backend.set_value(self.model.optimizer.lr, self.initial_learning_rate)
    
    def on_train_batch_end(self, batch, logs = None):
        lr = self.initial_learning_rate * tf.pow(gamma, batch * self.power)
        tf.keras.backend.set_value(self.model.optimizer.lr, lr)
        print('for ', batch, ' lr set to ', lr) 

In [ ]:
initial_learning_rate = 0.0005
gamma = 0.98
power = 1/2085

## MODEL 1

In [ ]:
inputs = keras.Input(shape = (Pixel, Pixel, Entrainement))

conv1 = layers.Conv2D(filters = 32, kernel_size = (3,3), activation = "relu", padding = "same")(inputs)
conv1 = layers.Conv2D(filters = 32, kernel_size = (3,3), activation = "relu", padding = "same")(conv1)

maxpool1 = layers.MaxPooling2D(pool_size = (2, 2))(conv1)

conv2 = layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu", padding="same")(maxpool1)
conv2 = layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu", padding="same")(conv2)

maxpool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = layers.Conv2D(filters=128,kernel_size=(3,3),activation="relu", padding="same")(maxpool2)
conv3 = layers.Conv2D(filters=128,kernel_size=(3,3),activation="relu", padding="same")(conv3)

maxpool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = layers.Conv2D(filters=256,kernel_size=(3,3),activation="relu", padding="same")(maxpool3)
conv4 = layers.Conv2D(filters=256,kernel_size=(3,3),activation="relu", padding="same")(conv4)
drop4 = layers.Dropout(0.5)(conv4)

maxpool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = layers.Conv2D(filters=512,kernel_size=(3,3),activation="relu", padding="same")(maxpool4)
conv5 = layers.Conv2D(filters=512,kernel_size=(3,3),activation="relu", padding="same")(conv5)
drop5 = layers.Dropout(0.5)(conv5)


up6 = layers.Conv2D(256, 2, activation = 'relu', padding="same")(layers.UpSampling2D(size = (2,2))(drop5))
merge6 = layers.concatenate([drop4, up6], axis = 3)

conv6 = layers.Conv2D(256, 3, activation = 'relu', padding="same")(merge6)
conv6 = layers.Conv2D(256, 3, activation = 'relu', padding="same")(conv6)



up7 = layers.Conv2D(128, 2, activation = 'relu', padding="same")(layers.UpSampling2D(size = (2,2))(conv6))
merge7 = layers.concatenate([conv3, up7], axis = 3)
conv7 = layers.Conv2D(128, 3, activation = 'relu', padding="same")(merge7)
conv7 = layers.Conv2D(128, 3, activation = 'relu', padding="same")(conv7)



up8 = layers.Conv2D(64, 2, activation = 'relu', padding="same")(layers.UpSampling2D(size = (2,2))(conv7))
merge8 = layers.concatenate([conv2, up8], axis = 3)
conv8 = layers.Conv2D(64, 3, activation = 'relu', padding="same")(merge8)
conv8 = layers.Conv2D(64, 3, activation = 'relu', padding="same")(conv8)

up9 = layers.Conv2D(32, 2, activation = 'relu', padding="same")(layers.UpSampling2D(size = (2,2))(conv8))
merge9 = layers.concatenate([conv1,up9], axis = 3)
conv9 = layers.Conv2D(32, 3, activation = 'relu', padding="same")(merge9)
conv9 = layers.Conv2D(32, 3, activation = 'relu' , padding="same")(conv9)


conv10 = layers.Conv2D(1, 3, activation = 'relu', padding="same")(conv9)


model1 = keras.Model(inputs, conv10)
#adam1 = keras.optimizers.Adam(lr = 0.0005)
model1.compile(optimizer = 'Adam', loss = 'mae', metrics = ['mae'])
model1.summary()

In [ ]:
N_epochs = 10
my_callbacks = [LRA(model=model1, initial_learning_rate = initial_learning_rate, gamma = gamma, power = power)] 
history1 = model1.fit(X_train, Y_train1, batch_size = 32, epochs = N_epochs, callbacks = my_callbacks)

In [ ]:
# SAVE MODEL
model1.save(os.path.join(directory_save, 'saved_model', 'model_1'))

## MODEL 2

In [ ]:
filter_size = 8
kernel_size = 5
pool_size = 2

activ = layers.ReLU(threshold = 1)
inputs = keras.Input(shape=(Pixel, Pixel, Entrainement))  # 5 images en entrée

conv1 = layers.Conv2D(filters = 2 * filter_size, kernel_size = kernel_size, padding = "same")(inputs)
conv1 = activ(conv1)
conv1 = layers.Conv2D(filters = 2 * filter_size, kernel_size = kernel_size, padding = "same")(conv1)
conv1 = activ(conv1)
conv1 = layers.Conv2D(filters = 2 * filter_size, kernel_size = kernel_size, padding = "same")(conv1)
conv1 = activ(conv1)

maxpool1 = layers.MaxPooling2D(pool_size = pool_size)(conv1)

conv2 = layers.Conv2D(filters = 4 * filter_size, kernel_size = kernel_size, activation = "relu", padding = "same")(maxpool1)
conv2 = layers.Conv2D(filters = 4 * filter_size, kernel_size = kernel_size, activation = "relu", padding = "same")(conv2)
conv2 = layers.Conv2D(filters = 4 * filter_size, kernel_size = kernel_size, activation = "relu", padding = "same")(conv2)

maxpool2 = layers.MaxPooling2D(pool_size = pool_size)(conv2)

conv3 = layers.Conv2D(filters = 8 * filter_size, kernel_size = kernel_size, activation = "relu", padding = "same")(maxpool2)
conv3 = layers.Conv2D(filters = 8 * filter_size, kernel_size = kernel_size, activation = "relu", padding = "same")(conv3)
conv3 = layers.Conv2D(filters = 8 * filter_size, kernel_size = kernel_size, activation = "relu", padding = "same")(conv3)

maxpool3 = layers.MaxPooling2D(pool_size = pool_size)(conv3)

conv4 = layers.Conv2D(filters = 16 * filter_size, kernel_size = kernel_size, activation = "relu", padding = "same")(maxpool3)
conv4 = layers.Conv2D(filters = 16 * filter_size, kernel_size = kernel_size, activation = "relu", padding = "same")(conv4)
conv4 = layers.Conv2D(filters = 16 * filter_size, kernel_size = kernel_size, activation = "relu", padding = "same")(conv4)

drop4 = layers.Dropout(0.5)(conv4)

maxpool4 = layers.MaxPooling2D(pool_size = pool_size)(conv4)

conv5 = layers.Conv2D(filters = 32 * filter_size, kernel_size = kernel_size, activation = "relu", padding = "same")(maxpool4)
conv5 = layers.Conv2D(filters = 32 * filter_size, kernel_size = kernel_size, activation = "relu", padding = "same")(conv5)
conv5 = layers.Conv2D(filters = 32 * filter_size, kernel_size = kernel_size, activation = "relu", padding = "same")(conv5)

drop5 = layers.Dropout(0.5)(conv5)


up6 = layers.Conv2D(16 * filter_size, 2, activation = 'relu', padding="same")(layers.UpSampling2D(size = (2,2))(drop5))
merge6 = layers.concatenate([drop4, up6], axis = 3)

conv6 = layers.Conv2D(16 * filter_size, 3, activation = 'relu', padding="same")(merge6)
conv6 = layers.Conv2D(16 * filter_size, 3, activation = 'relu', padding="same")(conv6)
conv6 = layers.Conv2D(16 * filter_size, 3, activation = 'relu', padding="same")(conv6)


up7 = layers.Conv2D(8 * filter_size, 2, activation = 'relu', padding="same")(layers.UpSampling2D(size = (2,2))(conv6))
merge7 = layers.concatenate([conv3, up7], axis = 3)
conv7 = layers.Conv2D(8 * filter_size, 3, activation = 'relu', padding="same")(merge7)
conv7 = layers.Conv2D(8 * filter_size, 3, activation = 'relu', padding="same")(conv7)
conv7 = layers.Conv2D(8 * filter_size, 3, activation = 'relu', padding="same")(conv7)


up8 = layers.Conv2D(4* filter_size, 2, activation = 'relu', padding="same")(layers.UpSampling2D(size = (2,2))(conv7))
merge8 = layers.concatenate([conv2, up8], axis = 3)
conv8 = layers.Conv2D(4 * filter_size, 3, activation = 'relu', padding="same")(merge8)
conv8 = layers.Conv2D(4 * filter_size, 3, activation = 'relu', padding="same")(conv8)
conv8 = layers.Conv2D(4 * filter_size, 3, activation = 'relu', padding="same")(conv8)

up9 = layers.Conv2D(2 * filter_size, 2, activation = 'relu', padding="same")(layers.UpSampling2D(size = (2,2))(conv8))
merge9 = layers.concatenate([conv1, up9], axis = 3)
conv9 = layers.Conv2D(2 * filter_size, 3, activation = 'relu', padding="same")(merge9)
conv9 = layers.Conv2D(2 * filter_size, 3, activation = 'relu' , padding="same")(conv9)
conv9 = layers.Conv2D(2 * filter_size, 3, activation = 'relu' , padding="same")(conv9)

 # 'Prediction' images en sortie, avec une activation Sigmoid pour sortir une proba
#conv10 = layers.Conv2D(Prediction, 3, activation = 'relu', padding="same")(conv9)  
conv10 = layers.Conv2D(1, 3, activation = 'relu', padding="same")(conv9)  # 1 seule prédiction

model2 = keras.Model(inputs, conv10)
model2.compile(optimizer = "Adam", loss = 'mae', metrics = ['mae'])
model2.summary()

In [ ]:
N_epochs = 10
my_callbacks = [LRA(model = model2, initial_learning_rate = initial_learning_rate, gamma = gamma, power = power)] 
history2 = model2.fit(X_train, Y_train1, batch_size = 32, epochs = N_epochs, callbacks = my_callbacks)

In [ ]:
# SAVE MODEL
model2.save(os.path.join(directory_save, 'saved_model', 'model_2'))

# MODEL 3

In [ ]:
filter_size = 8
kernel_size = 5
pool_size = 2

activ = layers.ReLU(threshold = 1)
inputs = keras.Input(shape=(Pixel, Pixel, Entrainement))  # 5 images en entrée

conv1 = layers.Conv2D(filters = 2 * filter_size, kernel_size = kernel_size, padding = "same")(inputs)
conv1 = activ(conv1)
conv1 = layers.Conv2D(filters = 2 * filter_size, kernel_size = kernel_size, padding = "same")(conv1)
conv1 = activ(conv1)
conv1 = layers.Conv2D(filters = 2 * filter_size, kernel_size = kernel_size, padding = "same")(conv1)
conv1 = activ(conv1)

maxpool1 = layers.MaxPooling2D(pool_size = pool_size)(conv1)

conv2 = layers.Conv2D(filters = 4 * filter_size, kernel_size = kernel_size, padding = "same")(maxpool1)
conv2 = activ(conv2)
conv2 = layers.Conv2D(filters = 4 * filter_size, kernel_size = kernel_size, padding = "same")(conv2)
conv2 = activ(conv2)
conv2 = layers.Conv2D(filters = 4 * filter_size, kernel_size = kernel_size, padding = "same")(conv2)
conv2 = activ(conv2)

maxpool2 = layers.MaxPooling2D(pool_size = pool_size)(conv2)

conv3 = layers.Conv2D(filters = 8 * filter_size, kernel_size = kernel_size, padding = "same")(maxpool2)
conv3 = activ(conv3)
conv3 = layers.Conv2D(filters = 8 * filter_size, kernel_size = kernel_size, padding = "same")(conv3)
conv3 = activ(conv3)
conv3 = layers.Conv2D(filters = 8 * filter_size, kernel_size = kernel_size, padding = "same")(conv3)
conv3 = activ(conv3)

maxpool3 = layers.MaxPooling2D(pool_size = pool_size)(conv3)

conv4 = layers.Conv2D(filters = 16 * filter_size, kernel_size = kernel_size, padding = "same")(maxpool3)
conv4 = activ(conv4)
conv4 = layers.Conv2D(filters = 16 * filter_size, kernel_size = kernel_size, padding = "same")(conv4)
conv4 = activ(conv4)
conv4 = layers.Conv2D(filters = 16 * filter_size, kernel_size = kernel_size, padding = "same")(conv4)
conv4 = activ(conv4)

drop4 = layers.Dropout(0.5)(conv4)

maxpool4 = layers.MaxPooling2D(pool_size = pool_size)(conv4)

conv5 = layers.Conv2D(filters = 32 * filter_size, kernel_size = kernel_size, padding = "same")(maxpool4)
conv5 = activ(conv5)
conv5 = layers.Conv2D(filters = 32 * filter_size, kernel_size = kernel_size, padding = "same")(conv5)
conv5 = activ(conv5)
conv5 = layers.Conv2D(filters = 32 * filter_size, kernel_size = kernel_size, padding = "same")(conv5)
conv5 = activ(conv5)


drop5 = layers.Dropout(0.5)(conv5)


up6 = layers.Conv2D(16 * filter_size, 2, padding="same")(layers.UpSampling2D(size = (2,2))(drop5))
up6 = activ(up6)
merge6 = layers.concatenate([drop4, up6], axis = 3)

conv6 = layers.Conv2D(16 * filter_size, 3, padding="same")(merge6)
conv6 = activ(conv6)
conv6 = layers.Conv2D(16 * filter_size, 3, padding="same")(conv6)
conv6 = activ(conv6)
conv6 = layers.Conv2D(16 * filter_size, 3, padding="same")(conv6)
conv6 = activ(conv6)

up7 = layers.Conv2D(8 * filter_size, 2, padding="same")(layers.UpSampling2D(size = (2,2))(conv6))
up7 = activ(up7)
merge7 = layers.concatenate([conv3, up7], axis = 3)
conv7 = layers.Conv2D(8 * filter_size, 3, padding="same")(merge7)
conv7 = activ(conv7)
conv7 = layers.Conv2D(8 * filter_size, 3, padding="same")(conv7)
conv7 = activ(conv7)
conv7 = layers.Conv2D(8 * filter_size, 3, padding="same")(conv7)
conv7 = activ(conv7)

up8 = layers.Conv2D(4* filter_size, 2, padding="same")(layers.UpSampling2D(size = (2,2))(conv7))
up8 = activ(up8)
merge8 = layers.concatenate([conv2, up8], axis = 3)
conv8 = layers.Conv2D(4 * filter_size, 3, padding="same")(merge8)
conv8 = activ(conv8)
conv8 = layers.Conv2D(4 * filter_size, 3, padding="same")(conv8)
conv8 = activ(conv8)
conv8 = layers.Conv2D(4 * filter_size, 3, padding="same")(conv8)
conv8 = activ(conv8)


up9 = layers.Conv2D(2 * filter_size, 2, padding="same")(layers.UpSampling2D(size = (2,2))(conv8))
up9 = activ(up9)
merge9 = layers.concatenate([conv1, up9], axis = 3)
conv9 = layers.Conv2D(2 * filter_size, 3, padding="same")(merge9)
conv9 = activ(conv9)
conv9 = layers.Conv2D(2 * filter_size, 3, padding="same")(conv9)
conv9 = activ(conv9)
conv9 = layers.Conv2D(2 * filter_size, 3, padding="same")(conv9)
conv9 = activ(conv9)

 # 'Prediction' images en sortie, avec une activation Sigmoid pour sortir une proba
#conv10 = layers.Conv2D(Prediction, 3, activation = 'relu', padding="same")(conv9)  
conv10 = layers.Conv2D(1, 3, padding="same")(conv9)  # 1 seule prédiction
conv10 = activ(conv10)

model3 = keras.Model(inputs, conv10)
model3.compile(optimizer = "Adam", loss = 'mae', metrics = ['mae'])
model3.summary()

In [ ]:
N_epochs = 10
my_callbacks =[LRA(model = model3, initial_learning_rate = initial_learning_rate, gamma = gamma, power = power)] 
history3 = model3.fit(X_train, Y_train1, batch_size = 32, epochs = N_epochs, callbacks = my_callbacks)

In [ ]:
# SAVE MODEL
model3.save(os.path.join(directory_save, 'saved_model', 'model_3'))

# PLOT THE RESULTS

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
epoch = list(range(1, N_epochs + 1))

def plot_history(history):
    fig, ax = plt.subplots()
    ax.plot(epoch, history.history['loss'], 'o-', label = "loss-mae")
    ax.set_xlabel("Epochs")
    ax.set_ylabel("mae loss")
    ax.legend()
    ax.set_title('Loss plot for epoch iteration')
    #plt.savefig("Loss_Train")

In [ ]:
plot_history(history1)



In [ ]:
plot_history(history2)


In [ ]:
plot_history(history3)


# PREPARE TEST DATA

In [ ]:
## LOOP FOR TEST DATA
init = 0
for p in part_test:
    pickle_f = load_fichier(year_test,p,1)
    print("This is the part " ,p)
    
    if pickle_f["miss_dates"].shape[0] == 0:
        print("There is no missing data in this chunk")
        data_radar, dates_radar = cut_timestep(pickle_f, Step_minute)
        print(data_radar.shape)
        data_cut = cut_data_and_coord(data_radar, Pixel, lat, lon)
        print( data_cut.shape) 
        X_TEST,Y_TEST = XYTRAIN(data_cut)
        print("Shape of the temporary train data", X_TEST.shape, Y_TEST.shape)     
        
    else:
        print("There is miss dates")
        data_radar,dates_radar = cut_timestep_miss2(pickle_f, Echeance_minute, Step_minute)
        print(data_radar.shape)
        data_cut = cut_data_and_coord(data_radar, Pixel, lat, lon)
        print(data_cut.shape)
        X_TEST, Y_TEST = XYTRAIN(data_cut)
        print("Shape of the temporary train data", X_TEST.shape, Y_TEST.shape)   
        
    if (init == 0):
        X_TEMP = X_TEST
        Y_TEMP = Y_TEST
        print("STEP N°1: ", X_TEMP.shape, Y_TEMP.shape)
    elif ( init == 1):
        X_test = np.append(X_TEMP, X_TEST, axis = 0)
        Y_test = np.append(Y_TEMP, Y_TEST, axis = 0)
        print("STEP N°2: ", X_test.shape, Y_test.shape)
    else:
        X_test = np.append(X_test, X_TEST, axis = 0)
        Y_test = np.append(Y_test, Y_TEST, axis = 0)
        print("OTHER STEP : ", X_test.shape, Y_test.shape)
    init += 1
    print("----------------------------------------------------------------------")

print("End of Test")
del(X_TEST, Y_TEST, data_cut, data_radar) 

In [ ]:
# VARIABLE FOR TEST DATA
year_test = 2018
month_test = [1]
part_test = [1, 2]

# MODEL EVALUATION

In [ ]:
Y_predict_models = ['','','']

In [ ]:
def model_evaluate(model_n):
    for k in range(3): # we will predict 3 images  
        Y_test_k = Y_test[:, :, :, k : k + 1]  # predict only 1 image at one time
        if k > 0:
            X_test_k = X_test_k[:, :, :, 1:]
            # remove first input image and add the previous predict image at the end of the input
            X_test_k = np.concatenate([X_test_k, Y_predict_k[:, :, :, 0:]], -1) 
        else:
            X_test_k = X_test[:, :, :, :]    
    
        results = model_n.evaluate(X_test_k, Y_test_k, batch_size = 20)    
        print("test loss, test acc:", results)    
    
        Y_predict_k = model_n.predict(X_test_k)  
   
        if k > 0:
            Y_predict_tot = np.concatenate([Y_predict_tot, Y_predict_k], -1)
        else:
            Y_predict_tot = Y_predict_k[:, :, :, :] 
    
    return Y_predict_tot

In [ ]:
# MODEL 1

Y_predict_models[0] = model_evaluate(model1)


In [ ]:
# MODEL 2

Y_predict_models[1] = model_evaluate(model2)


In [ ]:
# MODEL 3

Y_predict_models[2] = model_evaluate(model3)


In [ ]:
# SAVE Test and Predictions
#np.savez(os.path.join(directory_save, 'np_xtest_ytest_ypredictmodels'), X_test = X_test, Y_test = Y_test, Y_predict_models = Y_predict_models)

# MAP PREDICTIONS VISUALIZATIONS

In [ ]:
def LATLON_CUT(lat,lon):
    lat_edge = 49.5
    lon_edge = -2.5
    lat_only = lat[:,1]
    lon_only = lon[1,:]
    Temp_lat = np.where(lat_only < lat_edge)
    Temp_lon = np.where(lon_only < lon_edge)
    Lat_cut_index = Temp_lat[0][:Pixel]
    Lon_cut_index = Temp_lon[0][:Pixel]
    lat_format= lat[Lat_cut_index, Lon_cut_index]
    lon_format= lon[Lat_cut_index, Lon_cut_index]
    return lat_format, lon_format

In [ ]:
# Lat/Lon formating for plot.
lat_format, lon_format = LATLON_CUT(lat, lon)

In [ ]:
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib import colors
import cartopy.feature as cfeature
import numpy as np

In [ ]:
lllat = lat_format[-1]  #lower left latitude
urlat = lat_format[0]  #upper right latitude
lllon =  lon_format[0]  #lower left longitude
urlon =lon_format[-1]  #upper right longitude
extent = [lllon, urlon, lllat, urlat]

# Choose the colormap
cmap = colors.ListedColormap(['silver','white', 'darkslateblue', 'mediumblue','dodgerblue', 
                              'skyblue','olive','mediumseagreen','cyan','lime','yellow',
                              'khaki','burlywood','orange','brown','pink','red','plum'])
bounds = [-1, 0, 2, 4, 6, 8, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 75]
norm = colors.BoundaryNorm(bounds, cmap.N)


def map_predict(model_n, sequence):
    lats, lons = np.meshgrid(lat_format, lon_format)
    projection = ccrs.PlateCarree()
    axes_class = (GeoAxes, dict(map_projection = projection))
    fig = plt.figure(figsize = (80, 80))
    axgr = AxesGrid(fig, 222, axes_class = axes_class,
                    nrows_ncols = (2, 9),
                    axes_pad = 0.4,
                    cbar_location ='right',
                    cbar_mode = 'single',
                    cbar_size = '5%',
                    label_mode = '')
                   # shared_all=True)  # note the empty label_mode
    
    for i, ax in enumerate(axgr):
        ax.coastlines(resolution = '50m', linewidth = 2)
        ax.gridlines()
        ax.add_feature(cfeature.BORDERS.with_scale('50m'))
        if i < 6:
            p = ax.imshow(X_test[sequence, :, :, i], cmap=cmap, norm=norm, interpolation='none', origin='upper', extent=extent)
            ax.set_title('t - {} min'.format(75 - 15 * i))
        elif i >= 6 and i < 9:
            p = ax.imshow(Y_test[sequence, :, :, i - 6], cmap=cmap, norm=norm, interpolation='none', origin='upper', extent=extent)
            ax.set_title('truth : t + {} min'.format(15 * (i - 5)))
        elif i >= 9 and i < 15:
            p = ax.imshow(X_test[sequence, :, :, i - 9], cmap=cmap, norm=norm, interpolation='none', origin='upper', extent=extent)
        elif i >=15 and i < 19:
            p =  ax.imshow(model_n[sequence, :, :, i - 15], cmap=cmap, norm=norm, interpolation='none', origin='upper', extent=extent)
            ax.set_title('predict : t + {} min'.format(15 * (i - 14)))
    plt.colorbar(p, cmap=cmap, norm=norm, boundaries=bounds, ticks=bounds, cax=axgr.cbar_axes[0]).set_label('Rainfall (in 1/100 mm) / -1 : missing values')
    plt.show()

## MODEL 1

In [ ]:
map_predict(Y_predict_models[0], 92)

## MODEL 2

In [ ]:
map_predict(Y_predict_models[1], 92)

## MODEL 3

In [ ]:
map_predict(Y_predict_models[2], 92)


## ALL THE MODELS

In [ ]:
lllat = lat_format[-1]  #lower left latitude
urlat = lat_format[0]  #upper right latitude
lllon =  lon_format[0]  #lower left longitude
urlon =lon_format[-1]  #upper right longitude
extent = [lllon, urlon, lllat, urlat]

# Choose the colormap
cmap = colors.ListedColormap(['silver','white', 'darkslateblue', 'mediumblue','dodgerblue', 
                              'skyblue','olive','mediumseagreen','cyan','lime','yellow',
                              'khaki','burlywood','orange','brown','pink','red','plum'])
bounds = [-1, 0, 2, 4, 6, 8, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 75]
norm = colors.BoundaryNorm(bounds, cmap.N)

lats, lons = np.meshgrid(lat_format, lon_format)

projection = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(map_projection = projection))

fig = plt.figure(figsize = (80, 80))

axgr = AxesGrid(fig, 222, axes_class = axes_class,
                    nrows_ncols = (4, 9),
                    axes_pad = 0.4,
                    cbar_location ='right',
                    cbar_mode = 'single',
                    cbar_size = '5%',
                    label_mode = '')
                   # shared_all=True)  # note the empty label_mode


sequence = 92
for i, ax in enumerate(axgr):    
    ax.coastlines(resolution = '50m', linewidth = 2)
    ax.gridlines()
    ax.add_feature(cfeature.BORDERS.with_scale('50m'))

    if i % 9 < 6:
        p = ax.imshow(X_test[sequence, :, :, i % 9], cmap = cmap, norm = norm, interpolation = 'none', origin = 'upper', extent = extent)
        if i // 9 == 0:
            ax.set_title('t - {} min'.format(75 - 15 * i))
    else:
        if i // 9 == 0:
            p = ax.imshow(Y_test[sequence, :, :, i - 6], cmap = cmap, norm = norm, interpolation = 'none', origin = 'upper', extent = extent)
            ax.set_title('truth : t + {} min'.format(15 * (i - 5)))
        else:
            p =  ax.imshow(Y_predict_models[i//9 - 1][sequence, :, :, i % 9 - 6], cmap = cmap, norm = norm, interpolation = 'none', origin = 'upper', extent = extent)
            ax.set_title('Model {} : t + {} min'.format(i // 9, 15 * (i % 9 - 5)))
            
     
        
         

plt.colorbar(p, cmap=cmap, norm=norm, boundaries=bounds, ticks=bounds, cax=axgr.cbar_axes[0]).set_label('Rainfall (in 1/100 mm) / -1 : missing values')
plt.show()